In [1]:
import os
pwd = os.getcwd()

# Paths to sources
blaspp_source = "/home/weslleyp/storage/blaspp"
lapackpp_source = "/home/weslleyp/storage/lapackpp"
tlapack_source = "/home/weslleyp/storage/tlapack"

# Paths to libraries
tlapack_DIR = pwd+"/tlapack"
tlapackLAPACK_DIR = pwd+"/tlapack_LAPACK"
blaspp_DIR = pwd+"/blaspp"
lapackpp_DIR = pwd+"/lapackpp"
starpu_DIR = "/home/weslleyp/storage/starpu/build/lib"

# Set environment variables for StarPU
os.environ["PKG_CONFIG_PATH"] = starpu_DIR+"/pkgconfig"
os.environ["STARPU_SCHED"] = "dmdas"
os.environ["HWLOC_COMPONENTS"] = "-gl"

# Load Python modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import display, Math
import time

In [2]:
# System:
!uname -a

Linux weslleyp-XPS-15-9510 5.15.0-84-generic #93~20.04.1-Ubuntu SMP Wed Sep 6 16:15:40 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


In [3]:
# Machine:
!lscpu

Architecture:                       x86_64
CPU op-mode(s):                     32-bit, 64-bit
Byte Order:                         Little Endian
Address sizes:                      39 bits physical, 48 bits virtual
CPU(s):                             16
On-line CPU(s) list:                0-15
Thread(s) per core:                 2
Core(s) per socket:                 8
Socket(s):                          1
NUMA node(s):                       1
Vendor ID:                          GenuineIntel
CPU family:                         6
Model:                              141
Model name:                         11th Gen Intel(R) Core(TM) i7-11800H @ 2.30G
                                    Hz
Stepping:                           1
CPU MHz:                            2300.000
CPU max MHz:                        4600.0000
CPU min MHz:                        800.0000
BogoMIPS:                           4608.00
Virtualization:                     VT-x
L1d cache:                          384 KiB
L1i 

In [4]:
# Nvidia version:
!nvidia-smi

Mon Sep 25 15:44:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   63C    P0    18W /  45W |   1279MiB /  4096MiB |     34%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# MKL version:
!which mkl_link_tool

/opt/intel/oneapi/mkl/2023.2.0/bin/intel64/mkl_link_tool


In [16]:
#Build

# Install <T>LAPACK
!rm -rf "$tlapack_DIR"
!cmake -B "$tlapack_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D BUILD_EXAMPLES=OFF -D BUILD_TESTING=OFF -D TLAPACK_NDEBUG=ON -D CMAKE_INSTALL_PREFIX="$tlapack_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" "$tlapack_source"
!cmake --build "$tlapack_DIR" --target install

# Build
!rm -rf build
!cmake -B build -G Ninja -D CMAKE_BUILD_TYPE=Release -D CMAKE_PREFIX_PATH="."
!cmake --build build

-- The CXX compiler identification is GNU 9.4.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- <T>LAPACK version 0.1.1
-- Submodule update
-- Found Doxygen: /usr/local/bin/doxygen (found version "1.9.8 (396688347198e076597050259f4835853bdc3b44)") found components: doxygen dot 
-- ClangFormat found: /usr/bin/clang-format-10 (version 10.0.0)
-- Configuring done
-- Generating done
-- Build files have been written to: /home/weslleyp/storage/tlapack/examples/starpu/tlapack
[2/3] Install the project...nfig/versionrsion.dir/version.cpp.o
-- Install configuration: "Release"
-- Installing: /home/weslleyp/storage/tlapack/examples/starpu/tlapack/include
-- Installing: /home/weslleyp/storage/tlapack/examples/starpu/tlapack/include/tlapack copy.hpp
-- Installing: /home/weslleyp/storage/tla

In [17]:
!./build/example_starpu_potrf 100 20 all yes

n = 100
nb = 20
nt = 20

[starpu][weslleyp-XPS-15-9510][_starpu_init_cuda_config] Warning: could not find location of CUDA0, do you have the hwloc CUDA plugin installed?

-----------------------------------------------
float:
-----------------------------------------------
A = starpu::Matrix<f>( nrows = 100, ncols = 100 )
B = starpu::Matrix<f>( nrows = 100, ncols = 100 )
A = starpu::Matrix<f>( nrows = 100, ncols = 100 )
U^H U R = A   =>   ||R-Id||_1 / ||Id||_1 = 3.2332e-09
time = 8.54754 s

-----------------------------------------------
double:
-----------------------------------------------
A = starpu::Matrix<d>( nrows = 100, ncols = 100 )
^C


In [ ]:
!./build/example_starpu_lapack 1000

In [10]:
#Build with MKL

# Install BLAS++
!cmake -B "$blaspp_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D build_tests=OFF -D CMAKE_INSTALL_PREFIX="$blaspp_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" "$blaspp_source"
!cmake --build "$blaspp_DIR" --target install

# Install LAPACK++
!cmake -B "$lapackpp_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D build_tests=OFF -D CMAKE_INSTALL_PREFIX="$lapackpp_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" -D blaspp_DIR="$blaspp_DIR" "$lapackpp_source"
!cmake --build "$lapackpp_DIR" --target install

# Install <T>LAPACK
!cmake -B "$tlapackLAPACK_DIR" -G Ninja -D CMAKE_BUILD_TYPE=Release -D BUILD_EXAMPLES=OFF -D BUILD_TESTING=OFF -D TLAPACK_NDEBUG=ON -D CMAKE_INSTALL_PREFIX="$tlapackLAPACK_DIR" -D CMAKE_INSTALL_MESSAGE="LAZY" -D TLAPACK_USE_LAPACKPP=ON -D blaspp_DIR="$blaspp_DIR" -D lapackpp_DIR="$lapackpp_DIR" "$tlapack_source"
!cmake --build "$tlapackLAPACK_DIR" --target install

# Build
!cmake -B build_LAPACK -G Ninja -D CMAKE_BUILD_TYPE=Release -D tlapack_DIR="$tlapackLAPACK_DIR" -D blaspp_DIR="$blaspp_DIR" -D lapackpp_DIR="$lapackpp_DIR"
!cmake --build build_LAPACK

CMake Error at CMakeLists.txt:8 (cmake_minimum_required):
  CMake 3.17 or higher is required.  You are running version 3.16.3


-- Configuring incomplete, errors occurred!
Error: could not find CMAKE_PROJECT_NAME in Cache
CMake Error at CMakeLists.txt:8 (cmake_minimum_required):
  CMake 3.17 or higher is required.  You are running version 3.16.3


-- Configuring incomplete, errors occurred!
Error: could not find CMAKE_PROJECT_NAME in Cache
-- The CXX compiler identification is GNU 9.4.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- <T>LAPACK version 0.1.1
-- Submodule update
CMake Error at CMakeLists.txt:191 (find_package):
  By not providing "Findlapackpp.cmake" in CMAKE_MODULE_PATH this project has
  asked CMake to find a package configuration file provided by "lapackpp",
 

In [ ]:
!./build_LAPACK/example_starpu_potrf 1000 20 all yes

# First test: Find optimal block size for a given matrix size

In [ ]:
# Set environment variables for StarPU
if 'STARPU_NCUDA' in os.environ:
    print("STARPU_NCUDA: ", os.environ['STARPU_NCUDA'])
    del os.environ['STARPU_NCUDA']
if 'STARPU_NCPU' in os.environ:
    print("STARPU_NCPU: ", os.environ['STARPU_NCPU'])
    del os.environ['STARPU_NCPU']

In [ ]:
n = 7680

# vector with integer factors of n
parts = [i for i in range(8,21) if n%i==0]
N = len(parts)

precision = ["single","double"]
NT = len(precision)

nRuns = 5

executable = [
    "build_LAPACK/example_starpu_potrf"
]
methods = [
    r"C++ templates + cuBLAS + cuSOLVER + MKL"
]
M = len(executable)

# Partition sizes
nb = np.divide(n,parts)

# FLOPs
FLOPs_mul = n * (((n*1.)/6+.5)*n+1./3)
FLOPs_add = n * (((n*1.)/6)*n-1./6)
FLOPs = FLOPs_mul + FLOPs_add

print("Size of the matrix = "+str(n))
print("Partitions: "+str(parts))
print("Precisions: "+str(precision))
print("Number of runs = "+str(nRuns))

In [ ]:
expr = executable[0]
!$expr {n} {nb[3]} {precision[0]}

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

data = np.ones([M,N,NT], dtype=np.float64) * 60 * 60 * 24

for s in range(M):
    expr = executable[s]
    for i in range(N):
        nbi = nb[i]
        for k in range(NT):
            for j in range(nRuns):
                time.sleep(5)
                output = !$expr {n} {nbi} {precision[k]} | grep time
                print(output)
                try:
                    aux = float(output[-1].split()[2])
                    data[s,i,k] = np.minimum( aux, data[s,i,k] )
                except:
                    print("Not a float: ", output)

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

for p in range(NT):
    print(precision[p])

    fig1, ax1 = plt.subplots()

    for m in range(M):
        gflops = np.divide(FLOPs/1e9,data[m,:,p])
        plt.plot(nb,gflops,markers[m%3],label=methods[m])

    # ax1.set_xscale("log")
    # ax1.set_yscale("log")
    # ax1.set_xticks(parts)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("$n_b$")
    plt.ylabel("GFLOPS")
    plt.legend()

    plt.tight_layout()
    plt.savefig("curves_"+precision[p]+".pdf")
    plt.show()

# Take the best block size and compare with MKL and StarPU

In [ ]:
nx_best = np.zeros([NT], dtype=int)
for k in range(NT):
    nx_best[k] = parts[np.argmin(data[0,:,k])]
    # nx_best[k] = 16

print(nx_best)

nRuns = 5 # for the best partition size

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

dataMKL = np.ones([NT], dtype=np.float64) * 60 * 60 * 24

expr = "build/example_starpu_lapack"
for k in range(NT):
    for j in range(nRuns):
        time.sleep(5)
        output = !$expr {n} {precision[k]} | grep time
        print(output)
        try:
            aux = float(output[-1].split()[2])
            dataMKL[k] = np.minimum( aux, dataMKL[k] )
        except:
            print("Not a float: ", output)

for p in range(NT):
    print(precision[p])

    gflops = np.divide(FLOPs/1e9,dataMKL[p])
    print("time[s] = ", dataMKL[p])
    print("GFLOPS = ", gflops)

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

dataTLAPACKStarPUmkl = np.ones([NT], dtype=np.float64) * 60 * 60 * 24

expr = "build_LAPACK/example_starpu_potrf"
for k in range(NT):
    for j in range(nRuns):
        time.sleep(5)
        output = !$expr {n} {n/nx_best[k]} {precision[k]} | grep time
        print(output)
        try:
            aux = float(output[-1].split()[2])
            dataTLAPACKStarPUmkl[k] = np.minimum( aux, dataTLAPACKStarPUmkl[k] )
        except:
            print("Not a float: ", output)

for p in range(NT):
    print(precision[p])

    gflops = np.divide(FLOPs/1e9,dataTLAPACKStarPUmkl[p])
    print("time[s] = ", dataTLAPACKStarPUmkl[p])
    print("GFLOPS = ", gflops)

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

expr = starpu_DIR + "/starpu/examples/cholesky_implicit"
for j in range(nRuns):
    time.sleep(5)
    output = !$expr -size {n} -nblocks {nx_best[0]} -no-prio
    print(output[-1])

# First test: Find optimal block size without GPU

In [ ]:
# Set environment variables for StarPU
os.environ["STARPU_NCUDA"] = "0"
if 'STARPU_NCPU' in os.environ:
    del os.environ['STARPU_NCPU']

In [ ]:
n = 7680

# vector with integer factors of n
parts = [i for i in range(6,60) if n%i==0]
N = len(parts)

precision = ["single"]
NT = len(precision)

nRuns = 5

executable = [
    "build_LAPACK/example_starpu_potrf"
]
methods = [
    r"C++ templates + MKL"
]
M = len(executable)

# Partition sizes
nb = np.divide(n,parts)

# FLOPs
FLOPs_mul = n * (((n*1.)/6+.5)*n+1./3)
FLOPs_add = n * (((n*1.)/6)*n-1./6)
FLOPs = FLOPs_mul + FLOPs_add

print("Size of the matrix = "+str(n))
print("Partitions: "+str(parts))
print("Precisions: "+str(precision))
print("Number of runs = "+str(nRuns))

In [ ]:
expr = executable[0]
!$expr {n} {nb[0]} {precision[0]} y

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

data = np.ones([M,N,NT], dtype=np.float64) * 60 * 60 * 24

for s in range(M):
    expr = executable[s]
    for i in range(N):
        nbi = nb[i]
        for k in range(NT):
            for j in range(nRuns):
                time.sleep(5)
                output = !$expr {n} {nbi} {precision[k]} | grep time
                print(output)
                try:
                    aux = float(output[-1].split()[2])
                    data[s,i,k] = np.minimum( aux, data[s,i,k] )
                except:
                    print("Not a float: ", output)

In [ ]:
markers = ['x-','*-','+-']
plt.rcParams['font.size'] = 12

for p in range(NT):
    print(precision[p])

    fig1, ax1 = plt.subplots()

    for m in range(M):
        gflops = np.divide(FLOPs/1e9,data[m,:,p])
        plt.plot(nb,gflops,markers[m%3],label=methods[m])

    # ax1.set_xscale("log")
    # ax1.set_yscale("log")
    # ax1.set_xticks(parts)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

    plt.xlabel("$n_b$")
    plt.ylabel("GFLOPS")
    plt.legend()

    plt.tight_layout()
    plt.savefig("curves_"+precision[p]+".pdf")
    plt.show()

In [ ]:
nx_best = np.zeros([NT], dtype=np.float64)
for k in range(NT):
    nx_best[k] = parts[np.argmin(data[0,:,k])]
    # nx_best[k] = 15
print("Best nb:", np.divide(n,nx_best))

nRuns = 5 # for the best partition size

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

dataTLAPACKStarPUmkl = np.ones([NT], dtype=np.float64) * 60 * 60 * 24

expr = "build_LAPACK/example_starpu_potrf"
for k in range(NT):
    for j in range(nRuns):
        time.sleep(5)
        output = !$expr {n} {n/nx_best[k]} {precision[k]} | grep time
        print(output)
        try:
            aux = float(output[-1].split()[2])
            dataTLAPACKStarPUmkl[k] = np.minimum( aux, dataTLAPACKStarPUmkl[k] )
        except:
            print("Not a float: ", output)

for p in range(NT):
    print(precision[p])

    gflops = np.divide(FLOPs/1e9,dataTLAPACKStarPUmkl[p])
    print("time[s] = ", dataTLAPACKStarPUmkl[p])
    print("GFLOPS = ", gflops)

In [ ]:
# %%script false --no-raise-error
# # Comment the line above to run this cell

expr = starpu_DIR + "/starpu/examples/cholesky_implicit"
for j in range(nRuns):
    time.sleep(5)
    output = !$expr -size {n} -nblocks {nx_best[0]} -no-prio
    print(output[-1])

In [ ]:
expr = "build_LAPACK/example_starpu_potrf"
!$expr {n} {n/48} single